# The Battle of Neighborhoods 

## Problem and Background
Moving is never easy. In order to decide, to which city we are moving, there are plenties of factors to be accouted into consideration. One important aspect is, what kinds of venues are the most in each city. This could indirectly describe the cities. Someone who enjoy delicious food would rather choose a city, where there are many restaurants, while others who live a night life prefer a city with more pubs. This project aims at top 80 German cities in the term of populations and explore the cities in respect to venues . However it's quite time consuming if sometone lists all the venues for each city and summarize the data manuelly. As a data scientist I use Foursqare API to explore the top 50 venues including their categories for each city, afterwards I would take the information of the top 50 venues as features of every city and apply cluster analyse on these cities. This should reveals which cities are similar with each other, and which cities are quite different when it comes to nearby venues. The target group of this project is anyone, who tends to move to a German city and needs advice to narrow down their choices. To get a better view, some example venue categories as listed as follows:

   * Hotel
   * Cafe
   * Bar
   * Restaurant
   * Speakeasy
   * Historic sites
   * Garden

## Data Description
A list of needed data for this project:

   * List of City Name
   * List of City Latitude
   * List of City Lontitude
   * Group of Venues in each cities
   * Categories of every venue
   
As described in the section "Problem and Background", this notebook explore the top 80 German cities with the most populations. These cities can be founded directly in the wikipedia site: *https://en.wikipedia.org/wiki/List_of_cities_in_Germany_by_population*. Unfortunately, the table found in the wikipedia site does not provide any information about the geographical coordinates of each city. This problem can be solved by introducing the python library **geopy**, which can be used to locate every city. With geopy, we can easily map the cities into geographical coordinates. Now we use FourSquare API to explore each city, providing the geographical coordinates from the previous step. However, not all details of the venues matters in this project. In the phase of data preparation, we need extract the interesting information and transform them into the correct format. The most important information of each venue is the cities they belong to and their categories: Are they restaurants, bars or historic sites? Afterwards we can summarize the information for each city, and get the frequency of every venue category in the each city. At the last, the frequency table can be used to do the cluster analyse, which helps us to see the similarities between the cities. The result of the clustering will be visualized with **folium** map.

### 1. Scrape the city names 

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
url_wiki = "https://en.wikipedia.org/wiki/List_of_cities_in_Germany_by_population"
result = requests.get(url_wiki).content
soup = BeautifulSoup(result, "html.parser")
city_table = soup.find('tbody')

In [3]:
pd.set_option("display.max_rows", 1000)

In [4]:
city_df = pd.DataFrame(columns = ["City", "State"])
for raw in city_table.find_all('tr'):
    if raw.find_all('th') != []:
        continue
    else:
        columns = raw.find_all('td')
        city = columns[1].text.rstrip('\n')
        if '(' in city:
            city = city[0:city.index('(')]
        state = columns[2].text.rstrip('\n')
        city_df = city_df.append(pd.Series({"City": city, "State": state}), ignore_index = True)
city_df.head()

,City,State
0,Berlin,Berlin
1,Hamburg,Hamburg
2,Munich,Bavaria
3,Cologne,North Rhine-Westphalia
4,Frankfurt am Main,Hesse


### 2. Query Geographical Coordinates

In [5]:
# Query the geographical coordinates using geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="foursquare_agent")
latitudes = []
longitudes = []
for city in city_df.City:
    location = geolocator.geocode(city)
    latitude = latitudes.append(location.latitude)
    longitude = longitudes.append(location.longitude)
city_df["Latitude"] = latitudes
city_df["Longitude"] = longitudes

In [7]:
# sort the cities alphabetically
city_df.sort_values("City",inplace = True, ignore_index = True)
city_df.head()

,City,State,Latitude,Longitude
0,Aachen,North Rhine-Westphalia,50.776351,6.083862
1,Bergisch Gladbach,North Rhine-Westphalia,50.992930,7.127738
2,Bielefeld,North Rhine-Westphalia,52.019101,8.531007
3,Bremerhaven,Bremen,53.552226,8.586551
4,Darmstadt,Hesse,49.872775,8.651177


In [8]:
city_df.shape

(80, 4)

In [10]:
# Visualize the cities using Folium
import folium

German_location = [51.1642, 10.4541]
German_map = folium.Map(location = German_location, zoom_start = 6)
for city, latitude, longitude in zip(city_df['City'], city_df['Latitude'], city_df['Longitude']):
    folium.CircleMarker(
        location = [latitude, longitude],
        popup = city,
        radius = 3,
        color = 'blue',
        fill_color = 'blue'
        ).add_to(German_map)
German_map

### 3. Explore the cities using FourSquare API

In [13]:
# Credencials are blanked out for review
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = ''
LIMIT = 100
RADIUS = 2000

In [14]:
venue_df = pd.DataFrame(columns = ['city', 'VenueName', 'VenueCategory'])
for city, lat, lng in zip(city_df.City, city_df.Latitude, city_df.Longitude): 
    foursquare_url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}".format(
        CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, RADIUS, LIMIT)
    venues = requests.get(foursquare_url).json()['response']['groups'][0]['items']
    for venue in venues:
        venue_name = venue['venue']['name']
        venue_category = venue['venue']['categories'][0]['name']
        venue_df = venue_df.append(pd.Series({'city': city, 'VenueName': venue_name, 'VenueCategory': venue_category}), ignore_index = True)

In [15]:
venue_df.sort_values("city",inplace = True, ignore_index = True)
venue_df.head()

,city,VenueName,VenueCategory
0,Aachen,Vertical Weinbar,Wine Bar
1,Aachen,Hotel Europa,Music Venue
2,Aachen,Piccolino,Italian Restaurant
3,Aachen,HIT Sütterlin Tivoli,Supermarket
4,Aachen,Cafe Lammerskötter,Café


## Methology

### 1. Exploratory Data Analyse

For the first view of the cities. we can use the dataframe of the venues to group the venues by their belonging cities, and then calculate the frequency of the venue types in each city. At last, I'll list the top 5 most frequent venue types. In this way, we are able to see, what kinds of venues are most frequent in each city, which gives us an intuition of the cities.

In [16]:
# Onehot the categories for the purpose of later summary
venue_onehot = pd.get_dummies(venue_df['VenueCategory'], prefix="", prefix_sep="")

# Add belonging city to each venue
venue_onehot['city'] = venue_df['city']

# Make city as the first column
column_order = [venue_onehot.columns[-1]] + list(venue_onehot.columns[0:-1])
venue_onehot = venue_onehot[column_order]
venue_onehot.shape

(6292, 338)

In [17]:
venue_onehot.head()

,city,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Apple Wine Pub,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,...,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Aachen,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Aachen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aachen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Aachen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aachen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Group the venues by their cities and caculate the frequency of each venue in each city
venue_grouped = venue_onehot.groupby('city').mean().reset_index()
venue_grouped.shape

(80, 338)

In [20]:
venue_grouped.head()

,city,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Apple Wine Pub,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,...,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Aachen,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.010000,...,0.000000,0.00,0.0,0.01,0.000000,0.0,0.0,0.0,0.000000,0.0
1,Bergisch Gladbach,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.025641,...,0.000000,0.00,0.0,0.00,0.051282,0.0,0.0,0.0,0.000000,0.0
2,Bielefeld,0.0,0.00,0.0,0.0,0.0,0.0,0.011494,0.0,0.011494,...,0.011494,0.00,0.0,0.00,0.011494,0.0,0.0,0.0,0.011494,0.0
3,Bremerhaven,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.04,0.0,0.00,0.000000,0.0,0.0,0.0,0.020000,0.0
4,Darmstadt,0.0,0.01,0.0,0.0,0.0,0.0,0.000000,0.0,0.010000,...,0.000000,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0


In [21]:
# drop columns where the total frequency is too low

# This is because some venues use very special category names, which 
# won't be found anywhere else. This voids biase and reduces feature dimension

threshold = 0.03 # when the type shows not more than 3 times in all the venues
for column in venue_grouped.columns[1:]:
    if venue_grouped[column].sum() <= threshold:
        venue_grouped.drop(column, axis = 1, inplace = True)
venue_grouped.shape

(80, 208)

In [22]:
top_5_types = list()
for index in range(venue_grouped.shape[0]):
    freq = venue_grouped.iloc[index, 1:]
    top_freq = freq.sort_values(ascending = False).head(5)
    top_5_types.append(', '.join(top_freq.index.tolist()))

In [23]:
city_df["MostFrequentVenues"] = top_5_types
city_df.head()

,City,State,Latitude,Longitude,MostFrequentVenues
0,Aachen,North Rhine-Westphalia,50.776351,6.083862,"Supermarket, Bar, German Restaurant, Park, Café"
1,Bergisch Gladbach,North Rhine-Westphalia,50.992930,7.127738,"Supermarket, Drugstore, Shopping Mall, Wine Sh..."
2,Bielefeld,North Rhine-Westphalia,52.019101,8.531007,"Café, Supermarket, Bar, Middle Eastern Restaur..."
3,Bremerhaven,Bremen,53.552226,8.586551,"Supermarket, Hotel, Seafood Restaurant, Drugst..."
4,Darmstadt,Hesse,49.872775,8.651177,"Café, Supermarket, German Restaurant, Italian ..."


### 2. Cluster Analyse

In this part, I'll apply K Means Cluster Algorithmus to cluster these cities according to their venue frequencies. Cause the key problem of this project is to find the similarities between the cities, without any given labels. This categorizes our problem into unsupervised learning area. With K Means Clustering, we can discover the underlying structure of the data, and find the connections between the cities. 

In [65]:
from sklearn.cluster import KMeans
cluster_number = 4
kmeans_model = KMeans(n_clusters = cluster_number, init = 'k-means++', n_init = 10)
kmeans_model.fit(venue_grouped.iloc[:,1:])

KMeans(n_clusters=4)

## Result

In [66]:
city_df['Label'] = kmeans_model.labels_
city_df.head()

,City,State,Latitude,Longitude,MostFrequentVenues,Label
0,Aachen,North Rhine-Westphalia,50.776351,6.083862,"Supermarket, Bar, German Restaurant, Park, Café",2
1,Bergisch Gladbach,North Rhine-Westphalia,50.992930,7.127738,"Supermarket, Drugstore, Shopping Mall, Wine Shop, Clothing Store",3
2,Bielefeld,North Rhine-Westphalia,52.019101,8.531007,"Café, Supermarket, Bar, Middle Eastern Restaurant, Mediterranean Restaurant",2
3,Bremerhaven,Bremen,53.552226,8.586551,"Supermarket, Hotel, Seafood Restaurant, Drugstore, Restaurant",3
4,Darmstadt,Hesse,49.872775,8.651177,"Café, Supermarket, German Restaurant, Italian Restaurant, Drugstore",2


In [67]:
import matplotlib.cm as cm
import matplotlib.colors as colors

colors_array = cm.rainbow(np.linspace(0, 1, cluster_number))
rainbow = [colors.rgb2hex(i) for i in colors_array]

German_location = [51.1642, 10.4541]
German_map = folium.Map(location = German_location, zoom_start = 6)
for city, latitude, longitude, label, venue_freq in zip(city_df['City'], city_df['Latitude'], 
                                            city_df['Longitude'], city_df['Label'], city_df['MostFrequentVenues']):
    folium.CircleMarker(
        location = [latitude, longitude],
        popup = city + ': ' + venue_freq,
        radius = 3,
        color = rainbow[label],
        fill_color = 'blue'
        ).add_to(German_map)
German_map

### Interpret the Clusters

In [69]:
# Extract the top 5 venue types including their frequencies for each center point, which describes the charateristics of each cluster

cluster_centers = kmeans_model.cluster_centers_
centers_df = pd.DataFrame(cluster_centers, columns = venue_grouped.columns[1:])
centers_df = centers_df.T
centers_df.columns = ["Cluster 0", "Cluster 1", "Cluster 2", "Cluster 3"]
for cluster in range(centers_df.shape[1]):
    print(centers_df.iloc[:,cluster].sort_values(ascending = False).head(5))
    print('\n')

Café                  0.073268
Hotel                 0.058042
Italian Restaurant    0.036044
Coffee Shop           0.032331
German Restaurant     0.030103
Name: Cluster 0, dtype: float64


Gym / Fitness Center        0.222222
Supermarket                 0.111111
Fast Food Restaurant        0.111111
Mediterranean Restaurant    0.111111
German Restaurant           0.111111
Name: Cluster 1, dtype: float64


Supermarket          0.075197
Café                 0.066648
Drugstore            0.055156
Hotel                0.042939
German Restaurant    0.042398
Name: Cluster 2, dtype: float64


Supermarket             0.150191
Drugstore               0.072603
Café                    0.051557
Fast Food Restaurant    0.050003
Clothing Store          0.045255
Name: Cluster 3, dtype: float64




In [70]:
# show the cities belonging to Cluster 0
city_df[city_df["Label"] == 0]

,City,State,Latitude,Longitude,MostFrequentVenues,Label
7,Hanover,Lower Saxony,52.374478,9.738553,"Café, Coffee Shop, Plaza, Sushi Restaurant, Supermarket",0
8,Heidelberg,Baden-Württemberg,49.409358,8.694724,"Hotel, Café, Ice Cream Shop, Plaza, Coffee Shop",0
15,Regensburg,Bavaria,49.019533,12.097487,"Hotel, German Restaurant, Café, Drugstore, Clothing Store",0
16,Wolfsburg,Lower Saxony,52.420559,10.786168,"Hotel, Italian Restaurant, Clothing Store, Drugstore, Supermarket",0
17,Erlangen,Bavaria,49.597816,11.003806,"Café, Hotel, Pizza Place, Drugstore, Clothing Store",0
33,Saarbrücken,Saarland,49.234362,6.996379,"German Restaurant, Hotel, Italian Restaurant, Café, Nightclub",0
40,Berlin,Berlin,52.517037,13.388860,"Hotel, Monument / Landmark, Concert Hall, Bookstore, Coffee Shop",0
42,Bonn,North Rhine-Westphalia,50.735851,7.100660,"Café, Italian Restaurant, Pub, Bar, Drugstore",0
45,Bremen,Bremen,53.075820,8.807165,"Hotel, Café, Drugstore, Italian Restaurant, Falafel Restaurant",0
47,Cologne,North Rhine-Westphalia,50.938361,6.959974,"Café, Italian Restaurant, Coffee Shop, Cocktail Bar, Pedestrian Plaza",0


In [71]:
# show the cities belonging to Cluster 1
city_df[city_df["Label"] == 1]

,City,State,Latitude,Longitude,MostFrequentVenues,Label
34,Salzgitter,Lower Saxony,52.150372,10.359315,"Gym / Fitness Center, Supermarket, Fast Food Restaurant, Mediterranean Restaurant, German Restaurant",1


In [72]:
# show the cities belonging to Cluster 2
city_df[city_df["Label"] == 2]

,City,State,Latitude,Longitude,MostFrequentVenues,Label
0,Aachen,North Rhine-Westphalia,50.776351,6.083862,"Supermarket, Bar, German Restaurant, Park, Café",2
2,Bielefeld,North Rhine-Westphalia,52.019101,8.531007,"Café, Supermarket, Bar, Middle Eastern Restaurant, Mediterranean Restaurant",2
4,Darmstadt,Hesse,49.872775,8.651177,"Café, Supermarket, German Restaurant, Italian Restaurant, Drugstore",2
5,Freiburg im Breisgau,Baden-Württemberg,47.996090,7.849400,"Café, German Restaurant, Drugstore, Bar, Supermarket",2
6,Halle,Saxony-Anhalt,51.482504,11.970545,"Café, Supermarket, Drugstore, Hotel, Sandwich Place",2
11,Mainz,Rhineland-Palatinate,50.001231,8.276251,"German Restaurant, Café, Supermarket, Coffee Shop, Drugstore",2
12,Mülheim an der Ruhr,North Rhine-Westphalia,51.427293,6.882919,"Supermarket, Hotel, Park, German Restaurant, Event Space",2
13,Offenbach am Main,Hesse,50.105500,8.761070,"Supermarket, Drugstore, Pizza Place, Hotel, Nightclub",2
14,Osnabrück,Lower Saxony,52.271960,8.047635,"Supermarket, Bar, Café, Nightclub, Hotel",2
18,Fürth,Bavaria,49.477263,10.989616,"Supermarket, Bakery, Café, Italian Restaurant, German Restaurant",2


In [73]:
# show the cities belonging to Cluster 3
city_df[city_df["Label"] == 3]

,City,State,Latitude,Longitude,MostFrequentVenues,Label
1,Bergisch Gladbach,North Rhine-Westphalia,50.992930,7.127738,"Supermarket, Drugstore, Shopping Mall, Wine Shop, Clothing Store",3
3,Bremerhaven,Bremen,53.552226,8.586551,"Supermarket, Hotel, Seafood Restaurant, Drugstore, Restaurant",3
9,Herne,North Rhine-Westphalia,51.538039,7.219985,"Supermarket, Drugstore, German Restaurant, Metro Station, Fast Food Restaurant",3
10,Krefeld,North Rhine-Westphalia,51.333120,6.562334,"Supermarket, Café, Drugstore, Clothing Store, Electronics Store",3
21,Hildesheim,Lower Saxony,52.152164,9.951305,"Supermarket, Drugstore, Café, Hotel, Big Box Store",3
24,Ludwigshafen am Rhein,Rhineland-Palatinate,49.470411,8.438157,"Supermarket, Clothing Store, Hotel, Café, Drugstore",3
26,Moers,North Rhine-Westphalia,51.451283,6.628430,"Supermarket, Fast Food Restaurant, Clothing Store, Café, Snack Place",3
30,Recklinghausen,North Rhine-Westphalia,51.614382,7.197855,"Supermarket, Italian Restaurant, Clothing Store, Fast Food Restaurant, Shopping Mall",3
31,Remscheid,North Rhine-Westphalia,51.179871,7.194354,"Big Box Store, Drugstore, Supermarket, Hardware Store, Insurance Office",3
32,Reutlingen,Baden-Württemberg,48.491951,9.211414,"Supermarket, Café, Drugstore, Bar, Italian Restaurant",3


### Anomaly Analyse

In [76]:
# in this area , the details about the only city in the cluster 1 will be investigated.
venue_df[venue_df["city"] == "Salzgitter"]

,city,VenueName,VenueCategory
2502,Salzgitter,Kerasus,Mediterranean Restaurant
2503,Salzgitter,Restaurant Europa,Italian Restaurant
2504,Salzgitter,Focus Cinemas,Multiplex
2505,Salzgitter,Neumanns,German Restaurant
2506,Salzgitter,McDonald's,Fast Food Restaurant
2507,Salzgitter,T&T Markt,Grocery Store
2508,Salzgitter,real GmbH,Supermarket
2509,Salzgitter,McFIT Fitnessstudio,Gym / Fitness Center
2510,Salzgitter,Fitness Lounge Salzgitter,Gym / Fitness Center


## Discussion

First of all, I extracted the centers of the clusters and showed the top 5 venue types of each center. The centers are very representative and can explain the meaning of each cluster very well. Based on the observation, each cluster can be summerized as follows:
   * Cluster 0: cafe, hotel and German/Italian restaurant are very frequent in this cluster
   * Cluster 1: gyms are much more than other venues (explanation will be followed)
   * Cluster 2: supermarkets, cafes and drugstores are a lot, and the differecnce of their frequencies are slight   
   * Cluster 3: supermarkets are dominant in this cluster (about twice more than the second venue type), which is follwed by drugstore, cafe, fast food restaurant and clothing store 
   
Afterwards, I showed the cities in each cluster, in order to validate the summary inferred from the cluter centers. In the column *MostFrequentVenues*, the venue types are showed according to their rankings. The most left venue type has the most frequency among these five venue types. In cluster 0, we can see lots of cafes, hotels and Italian/German restaurants in the lists, while supermarkets and drugstores show up much moch in cluster 2 than cluster 0. For cluster 3, Supermarkets are in the first place almost in each city, and drugstores, cafes, fast food restaurants and clothing stores can also been seen a lot. These three clusters accord with the interpretation above. However, the cluster 1 includes only one city - Salzgitter, which makes this cluster as its own cluster. Even though, it doesn't make much sense that gyms are much more. In ordder to see the details about the city, all venues in Salzgitter are listed in the section "Anomaly Analyse". It turns out that there are only 11 venues found in this city. It could be a limitation of the FourSquare API, that not many venues recorded in Salzgitter. In this case, we'll treat this city (respectively its cluster) as an anomaly point, and discard it from our analyse.

## Conclusion

Regarding the analyse above, some conclusions can be drawn drom the results:
   * Most cities in cluster 0 are popular German cities, which are frequently visited by tourists. This explains why there are so many restaurants and hotels in these vities. If someone like bustling cities, the cities in cluster 0 could be his choices.
   * The most frequent venues types from cluster 2 and cluster 3 are very similar. As discussed above, in cluster 3, the supermarkets are dominant, while hotels are not that frequent compared to other clusters. If someone enjoys a quiet life (for example, when someone buys groceries and cooks by himself more often than going to restaurants, or when someone does not like many tourists near his home), he can consider the cities in cluster 3.
   * Cluster 2 can be almost seen as the mixture of the other two clusters. There are many restaurants and hotels, meanwhile there are also lots of drugstores and supermarkets for the needs of the natives. Therefore cluster 2 are choices between cluster 0 and cluster 3.